In [49]:

import cobra as cb
import pandas as pd
from glob import glob
import re
import sys
import getopt
import os.path
import copy
import csv
import math
import cobra.flux_analysis.variability
import subprocess
import shutil, errno
import statistics
from cobra import Reaction
import cometspy as c
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile

In [3]:
#loading a sbml model

def initialize_models():
    if not(os.path.exists("ModelsInput/iJO1366.cmd")):
        print('ERROR! Not iJO1366.cmd files with GEM of consortium strains in ModelsInput!')
    else:
        #os.chdir("ModelsInput")
        model=c.model("ModelsInput/iJO1366.cmd")
        model_id=model.id
    return model, model_id #not sure if removing model and read it in initialize_layout, just like in ecolilongtermFlYCOP...

#Stablish the layout file 
def initialize_layout(model):
    layout=c.layout(model)
    return layout
    

In [155]:
def model1_modification(model_original,model_output,ac1,glu1,o21):
    model1=cb.io.read_sbml_model(model_original)
    model1.reactions.get_by_id('EX_glc__D_e').bounds=(-1000,1000)
    model1.reactions.get_by_id('EX_o2_e').bounds=(-1000,1000)
    model1.reactions.get_by_id('GLCtex_copy1').bounds=(0,-glu1)
    model1.reactions.get_by_id('O2tex').bounds=(0,-o21)
    if(ac1<=0):
        model1.reactions.get_by_id('ACtex').bounds=(-1000,-ac1)
        model1.reactions.get_by_id('EX_ac_e').bounds=(ac1,1000)
    else:
        model1.reactions.get_by_id('ACtex').bounds=(-ac1,-1000)
        model1.reactions.get_by_id('EX_ac_e').bounds=(-1000,ac1)
    final_model=c.model(model1)
    final_model.id=model_output
    final_model.write_comets_model()
    del(final_model)
    
def model2_modification(model_original,model_output,ac2,glu2,o22):
    
    model2=cb.io.read_sbml_model(model_original)
    model2.reactions.get_by_id('EX_glc__D_e').bounds=(-1000,1000)
    model2.reactions.get_by_id('EX_o2_e').bounds=(-1000,1000)
    model2.reactions.get_by_id('GLCtex_copy1').bounds=(0,-glu2)
    model2.reactions.get_by_id('O2tex').bounds=(0,-o22)
        
    if(ac2<=0):
        model2.reactions.get_by_id('ACtex').bounds=(-1000,-ac2)
        model2.reactions.get_by_id('EX_ac_e').bounds=(ac2,1000)    
    else:
        model2.reactions.get_by_id('ACtex').bounds=(-ac2,-1000)
        model2.reactions.get_by_id('EX_ac_e').bounds=(-1000,ac2)
    final_model=c.model(model2)
    final_model.id=model_output
    final_model.write_comets_model()        
    del(final_model)
    
        

In [ ]:
def draw_results(filename):


In [150]:
def get_uptakes(fluxes, mets_index, model_number):
    numRxnMet=mets_index
    uptakeMet=[]
    
    for i in range(0,model_number):
        uptake=fluxes.loc[(comets.fluxes[0]==5) & (fluxes[1]==1) & (fluxes[2]==1) & (fluxes[3]==i)][mets_index-1]
        uptakeMet.append(float(uptake))
    return(uptakeMet)

In [151]:
def ecolilongTermFLYCOP_oneConf(glu1,ac1,o21,glu2,ac2,o22, fitFunc='MaxYield_MinTime',dirPlot='',repeat=10, params_file=None, layout_file="Nuevo_layout.txt"):
    fitness=0
    #check if class model exist
    try:
        var = c.model()
    except NameError:
        #clas model doesn't exists
        model, model_id= initialize_models()
        #print(model)
    print("Fitness function:"+fitFunc)
    
  # Single GEMs parameter modifications
  # ===================================
    if not(os.path.exists('ecoli_1_tmp.cmd')):
        # 1.1.- [COBRApy] Establish modifications in model 1 
        model1_modification("iJO1366.xml",'ecoli_1_tmp',ac1,glu1,o21)
        # 1.2.- [COBRApy] Establish modifications in model 2
        model2_modification("iJO1366.xml",'ecoli_2_tmp',ac2,glu2,o22)
    
    # Reading layout
    layout=initialize_layout(layout_file)
    #establish params with a global params file
    params = c.params(global_params = params_file)
    
    #establish comets class
    comets = c.comets(layout, params)
    #comets.parameters.all_params['TotalBiomassLogName'] = 'total_biomass_log_template.txt'
    #comets.parameters.all_params['BiomassLogName'] = 'biomass_log_template.txt'
    #comets.parameters.all_params['FluxLogName'] = 'flux_log_template.txt'
    #comets.parameters.all_params['MediaLogName'] = 'media_log_template.txt'

    #comets.set_classpath('lang3', '/comets/lib/commons-lang3-3.9/commons-lang3-3.9.jar')
    #comets.set_classpath('gurobi', '/home/ana/gurobi903/linux64/lib/gurobi.jar')
    #comets.set_classpath('bin', '/home/ana/comets/bin/comets_2.10.0.jar')
    if not(os.path.exists('IndividualRunsResults')):
        os.makedirs('IndividualRunsResults')
    totfitness=0
    sumTotBiomass=0
    sumTotYield=0
    biomass1=0.01
    biomass2=0.01
    fitnessList=[]
      
    # To repeat X times, due to random behaviour in COMETS:
    for i in range(repeat):
        comets.run(delete_files=True)
        comets.total_biomass.to_csv('total_biomass_log_template.txt',sep='\t',index=False)
        comets.media.to_csv('media_log_template.txt',sep='\t',index=False)
        comets.fluxes.to_csv('flux_log_template.txt',sep='\t',index=False)

        with open("output.txt", "w") as f:
            f.write(comets.run_output)

        #graphic part
        #subprocess.call(['../../Scripts/plot_biomassX2_vs_2mediaItem.sh','template', 'glc_D_e','ca2_e','Ecoli1','Ecoli2'])
        biomassFilename='total_biomass_log_template.txt'
        mediaFilename='media_log_template.txt'
        met='glc__D_e'
        met2='ac_e'
        outfile1=[]
        outfile2=[]
        
        #Draw results
        with open(mediaFilename, 'r') as f:
            lines = f.readlines()
            #lines =[item.replace(",", ".") for item in lines]
            outfile1=[[str.split(line)[4]] * 5 for line in lines if met in line]
            outfile1 = [item for sublist in outfile1 for item in sublist]
            outfile1=outfile1[:-4]
            outfile2=[[str.split(line)[4]] * 5 for line in lines if met2 in line]
            outfile2 = [item for sublist in outfile2 for item in sublist]
            outfile2.append(outfile2[-1])

        with open(biomassFilename,'r') as g:
            g = pd.read_csv(g, delimiter='\t', decimal='.')
            df1 = g.iloc[:, 0:3]
            df1["met1"]=outfile1
            df1["met2"]=outfile2
            np.savetxt(r'biomass_vs_glc_D_e_ac_e_template.txt', df1.values, fmt='%s',delimiter='\t')

            df1.reset_index()
            df1=df1.astype(float)
            df1.columns=['time(h)','Ecoli1', 'Ecoli2', met, met2]
            plt.ioff()
            fig, ax = plt.subplots()
            ax.plot(df1['time(h)'], df1['Ecoli1'], label='Ecoli1')
            ax.plot(df1['time(h)'], df1['Ecoli2'], linestyle='--', label='Ecoli2')
            ax.plot(df1['time(h)'], df1[str(met)], label=str(met))
            ax.plot(df1['time(h)'], df1[str(met2)], label=str(met2))
            plt.legend()
            plt.savefig('biomass_vs_glc_D_e_ac_e_template_plot1.pdf')
            df1.drop(df1.index, inplace=True)
    
        # 7.- Compute fitness (measure to optimize):
        print('computing fitness...')
        # 7.1.- Determine endCycle: when glucose and acetate are exhausted
        with open("biomass_vs_glc_D_e_ac_e_template.txt", "r") as sources:
            lines = sources.readlines()
            iniPointV=lines[0].split()
            iniBiomass=float(iniPointV[1])+float(iniPointV[2])
            totGlc=float(iniPointV[3])
            endGlcCycle=0
            for line in lines:
                endCycle=int(float(line.split()[0]))
                glcConc=float(line.split()[3])
                acConc=float(line.split()[4])
                if((endGlcCycle==0)and(glcConc==0.0)):
                    endGlcCycle=endCycle
                if((glcConc==0.0)and(acConc==0.0)):
                    break;
                if((glcConc==0.0)and(ac1>=0)and(ac2>=0)):
                    break;
            endPointV=lines[endCycle].split()
             # 7.2.- Compute first element fitness: maximize biomass yield
        # To compute final biomass as the maximum biomass of each strain
            finalBiomass1=0
            finalBiomass2=0
            count=0
            for line in lines:
                if(float(line.split()[1])>finalBiomass1):
                    finalBiomass1=float(line.split()[1])
                if(float(line.split()[2])>finalBiomass2):
                    finalBiomass2=float(line.split()[2])
                if(count>endCycle):
                    break;
                count=count+1
            finalBiomass=finalBiomass1+finalBiomass2
            biomassYieldNew=float((finalBiomass-iniBiomass)/(totGlc**0.1801559)) # molecular weigth of met1 per mmol
            # For normalizing yield
            MaximumYield=0.6
            # 7.3.- Compute second element fitnes: minimize time        
            fitTime=1-(float(endCycle)/float(240))
            # 7.4.- Compute joint fitness, as a 50% each element.
            if(fitFunc=='MaxYield_MinTime'):
                fitness=0.5*(biomassYieldNew/MaximumYield)+0.5*fitTime #Normalizing yield
            elif fitFunc=='Yield':
                fitness=(biomassYieldNew/MaximumYield)
            if(float(finalBiomass-iniBiomass) > 1.03):  # Given that with both strains with WT, total biomass=1.028
                fitness=0
            
            fluxFilename = 'flux_log_template.txt'
            numRxnMet=37
            uptakeMet=get_uptakes(comets.fluxes, numRxnMet, len(layout.models))
            print(uptakeMet)
            
            #for i in range(1,len(layout.models)):
            #    pattern ="2 1 1 "+str(i)
            #    comets.fluxes.loc
            #    with open(fluxFilename, "r") as f:
            #        lines = f.readlines()
            #        for line in lines:
            #            if re.search(pattern, line):
            #                line.replace(pattern, '')
            #                numbers=list(line.split(" "))
            #                uptakeMet.append(float(numbers[numRxnMet-1]))        
            
            print(" Total biomass: "+str(round(finalBiomass,6))+" in cycle "+str(endCycle)+". Biomass yield="+str(round(biomassYieldNew,6)))
            totfitness=totfitness+fitness
            fitnessList.append(fitness)
            sumTotBiomass=sumTotBiomass+finalBiomass
            sumTotYield=sumTotYield+biomassYieldNew
            
             # Copy individual solution
            file='IndividualRunsResults/'+'biomass_vs_glc_D_ac_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'        
            shutil.move('biomass_vs_glc_D_e_ac_e_template_plot1.pdf',file)
            if(dirPlot != ''):
                file2=dirPlot+'biomass_vs_glc_D_ac_'+str(glu1)+'_'+str(ac1)+'_'+str(o21)+'_'+str(glu2)+'_'+str(ac2)+'_'+str(o22)+'_'+str(round(uptakeMet[0],1))+'_'+str(round(uptakeMet[1],1))+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'
                shutil.copy(file,file2)
                
            
            
            file='IndividualRunsResults/'+'total_biomass_log_run'+str(i)+'.txt'
            shutil.move(biomassFilename,file)
            file='IndividualRunsResults/'+'media_log_run'+str(i)+'.txt'
            shutil.move(mediaFilename,file)
            file='IndividualRunsResults/'+'flux_log_run'+str(i)+'.txt'
            shutil.move(fluxFilename,file)
            
    avgfitness=totfitness/repeat
    sdfitness=statistics.stdev(fitnessList)
    avgBiomass=sumTotBiomass/repeat
    avgYield=sumTotYield/repeat
    print("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle")
    print(fitFunc+"\t"+str(glu1)+','+str(ac1)+','+str(o21)+','+str(glu2)+','+str(ac2)+','+str(o22)+','+str(round(uptakeMet[0],1))+','+str(round(uptakeMet[1],1))+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle))
    with open(dirPlot+"configurationsResults"+fitFunc+".txt", "a") as myfile:
        myfile.write("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\n")
        myfile.write(fitFunc+"\t"+str(glu1)+','+str(ac1)+','+str(o21)+','+str(glu2)+','+str(ac2)+','+str(o22)+','+str(round(uptakeMet[0],1))+','+str(round(uptakeMet[1],1))+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\n")
    print("Avg.fitness(sd):\t"+str(avgfitness)+"\t"+str(sdfitness)+"\n")
    if(sdfitness>0.1):
        avgfitness=0.0
    return avgfitness,sdfitness



In [154]:
ecolilongTermFLYCOP_oneConf(-10,-18,-15,-10,-18,-15, fitFunc='Yield',dirPlot='',repeat=2, params_file="parametes.txt")

Fitness function:Yield


NameError: name 'ac1' is not defined

In [4]:
params_file="Parametes.txt"
layout_file="Nuevo_layout.txt"

In [8]:
layout=c.layout('Nuevo_layout.txt')
params = c.params(global_params = "parametes.txt")
comets = c.comets(layout, params)

In [56]:
test=comets.run(delete_files=False)


Running COMETS simulation ...
Done!


In [14]:
with open("output2.txt","w") as f:
    f.write(comets.run_output)


In [65]:
mediaFilename='media_log_template.txt'
met='glc__D_e'
met2='ac_e'
outfile1=[]
outfile2=[]
        
        #Draw results
with open(mediaFilename, 'r') as f:
    lines = f.readlines()
    lines =[item.replace(",", ".") for item in lines]
    outfile1=[[str.split(line)[4]] * 5 for line in lines if met in line]
    outfile1 = [item for sublist in outfile1 for item in sublist]
    outfile1=outfile1[:-4]
    outfile2=[[str.split(line)[4]] * 5 for line in lines if met2 in line]
    outfile2 = [item for sublist in outfile2 for item in sublist]
    outfile2.append(outfile2[-1])

In [71]:
outfile1

['10.0',
 '10.0',
 '10.0',
 '10.0',
 '10.0',
 '9.8846882395',
 '9.8846882395',
 '9.8846882395',
 '9.8846882395',
 '9.8846882395',
 '9.721978363',
 '9.721978363',
 '9.721978363',
 '9.721978363',
 '9.721978363',
 '9.4923876972',
 '9.4923876972',
 '9.4923876972',
 '9.4923876972',
 '9.4923876972',
 '9.1684253485',
 '9.1684253485',
 '9.1684253485',
 '9.1684253485',
 '9.1684253485',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157300365',
 '8.8157

In [75]:
g = pd.read_csv('total_biomass_log_template.txt', delimiter='\t', decimal='.')

In [84]:

df1 = g.iloc[:, 0:3]
df1[:,met]=outfile1
df1[:,met2]=outfile2

TypeError: unhashable type: 'slice'

In [95]:
comets.run(delete_files=True)


Running COMETS simulation ...
Done!


In [96]:
import locale

loc = locale.getlocale() 

1


In [127]:
comets.fluxes.loc[(comets.fluxes[0]==5) & (comets.fluxes[1]==1) & (comets.fluxes[2]==1) & (comets.fluxes[3]==1)][41]

0   -0.0
Name: 41, dtype: float64

In [119]:
comets.fluxes[0]

0       5
1       5
2      10
3      10
4      15
     ... 
91    230
92    235
93    235
94    240
95    240
Name: 0, Length: 96, dtype: int64

In [138]:
layout_file="Nuevo_layout.txt"
params_file="parametes.txt"
layout=initialize_layout(layout_file)
params = c.params(global_params = params_file,package_params=params_file)
comets = c.comets(layout, params)

In [141]:
comets.run(delete_files=False)
comets.fluxes


Running COMETS simulation ...
Done!


,0,1,2,3,4,5,6,7,8,9,...,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586
0,1,1,1,1,0.0,-0.0,7.953277,-0.000017,0.000159,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017684,0.018259,0.018259
1,1,1,1,2,0.0,-0.0,7.953277,-0.000017,0.000159,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017684,0.018259,0.018259
2,2,1,1,1,0.0,-0.0,7.953279,-0.000017,0.000159,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017684,0.018259,0.018259
3,2,1,1,2,0.0,-0.0,7.953279,-0.000017,0.000159,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017684,0.018259,0.018259
4,3,1,1,1,0.0,-0.0,7.953281,-0.000017,0.000159,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017684,0.018259,0.018259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,238,1,1,2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
476,239,1,1,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
477,239,1,1,2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
478,240,1,1,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
